In [67]:
import sys
import re
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

sys.path.append('../src')

import columnar as col

ROOT_PATH = '../'

plt.style.use('fivethirtyeight')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
df = pd.read_csv('../data/mushrooms/mushrooms.csv')
df['class_ep'] = df['class'].map({'p': 0, 'e': 1})
df.columns = [col.replace('-', '_') for col in df.columns]
df.drop(['class'], axis=1, inplace=True)



In [25]:
df.head().T

,0,1,2,3,4
age,73,58,18,9,10
class_of_worker,Not in universe,Self-employed-not incorporated,Not in universe,Not in universe,Not in universe
industry_code,0,4,0,0,0
occupation_code,0,34,0,0,0
adjusted_gross_income,High school graduate,Some college but no degree,10th grade,Children,Children
education,0,0,0,0,0
wage_per_hour,Not in universe,Not in universe,High school,Not in universe,Not in universe
enrolled_in_edu_inst_last_wk,Widowed,Divorced,Never married,Never married,Never married
marital_status,Not in universe or children,Construction,Not in universe or children,Not in universe or children,Not in universe or children
major_industry_code,Not in universe,Precision production craft & repair,Not in universe,Not in universe,Not in universe


In [ ]:
loader = col.DataLoader(task='census_income', root=ROOT_PATH)
df = loader.load_data()
display(df.sample(5))